In [1]:
from GlucoCheck.glucoCheck import glucoCheckOps
import pandas as pd
import random
import numpy as np
from tqdm.auto import tqdm

from sklearn.preprocessing import MinMaxScaler
from scipy import stats



import random
import re
from dateutil.parser import parse

import warnings  
warnings.filterwarnings('ignore')

import os
import math


Using TensorFlow backend.


In [2]:
def createGap(df,start,end):
    l = len(df.index)
    if end>=l:
        end = l
        
    for i in range(start,end):
        df['GlucoseValue'][i]= float("NaN")
    
    return df


In [3]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/Data/Ohio-Data/OhioFullConsolidated.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])
data = data[data['subjectId']=='OD552']
data = data.reset_index(drop=True)
data

,subjectId,Display Time,GlucoseValue
0,OD552,2025-04-16 11:17:00,95
1,OD552,2025-04-16 11:22:00,86
2,OD552,2025-04-16 11:27:00,81
3,OD552,2025-04-16 11:32:00,81
4,OD552,2025-04-16 11:37:00,82
...,...,...,...
11439,OD552,2025-06-07 16:49:00,238
11440,OD552,2025-06-07 16:54:00,233
11441,OD552,2025-06-07 16:59:00,229
11442,OD552,2025-06-07 17:04:00,224


In [4]:
# data.to_csv('~/Desktop/r.csv')

In [5]:
obj = glucoCheckOps()

In [8]:
seed = 9250
start = seed
end = seed+44

In [9]:
dataWithMissing = data.copy()
dataWithMissing = createGap(dataWithMissing,start,end)
# dataWithMissing['Display Time'] = pd.to_datetime(dataWithMissing['Display Time'])
# dataWithMissing = dataWithMissing.set_index(['Display Time'], drop=True)

In [10]:
imputed = obj.impute(dataWithMissing)

Gap < 50; We use LSTM imputations
Training Model...

Model trained successfully!
Imputation Successful


In [11]:
imputed

,subjectId,Display Time,GlucoseValue
0,OD552,2025-04-16 11:17:00,95.0
1,OD552,2025-04-16 11:22:00,86.0
2,OD552,2025-04-16 11:27:00,81.0
3,OD552,2025-04-16 11:32:00,81.0
4,OD552,2025-04-16 11:37:00,82.0
...,...,...,...
11439,OD552,2025-06-07 16:49:00,238.0
11440,OD552,2025-06-07 16:54:00,233.0
11441,OD552,2025-06-07 16:59:00,229.0
11442,OD552,2025-06-07 17:04:00,224.0


In [12]:
obj.classifier(imputed)

array(['diabetic'], dtype=object)